In [ ]:
!pip install timm
import sys
sys.path.append('../input/ttach-kaggle')
import ttach

In [ ]:
# ライブラリ読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import pickle
import time
import gc
import random
import copy
import math
from math import ceil
import sys
import cv2
import pandas_profiling as pdp
import warnings
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import ttach as tta
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict
from torch.optim import lr_scheduler
from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
import albumentations as A
import timm

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100) # 表示できる表の列数

In [ ]:
timm.list_models(pretrained=False)

In [ ]:
Config = {"seed": 14,
          "epochs": 300,
          "img_size": 256,
          "batch_size": 32,
          "learning_rate": 0.001,
          "scheduler": 'OneCycleLR',
          "min_lr": 1e-6,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "model_name": "resnet18d",
          "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
          }

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(Config['seed'])

In [ ]:
DATA_DIR = "../input/ai-medical-contest-2022/"
df_train = pd.read_csv(DATA_DIR + "train.csv")
df_test = pd.read_csv(DATA_DIR + "test.csv")
df_unlabeled = pd.read_csv(DATA_DIR + "unlabeled.csv")
df_sub = pd.read_csv(DATA_DIR + "sample_submission.csv")

In [ ]:
# 画像データのpathの列を追加.
df_train['path'] = df_train['id'].apply(lambda x: "../input/ai-medical-contest-2022/image/image/{}.png".format(x))
df_test['path'] = df_test['id'].apply(lambda x: "../input/ai-medical-contest-2022/image/image/{}.png".format(x))
df_unlabeled['path'] = df_unlabeled['id'].apply(lambda x: "../input/ai-medical-contest-2022/image/image/{}.png".format(x))

In [ ]:
skf = StratifiedKFold(n_splits=Config['n_fold'], random_state=Config['seed'], shuffle=True)

for fold, ( _, val_) in enumerate(skf.split(X=df_train, y=df_train['pneumonia'])):
      df_train.loc[val_ , "kfold"] = fold

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['path'].values
        self.labels = df['pneumonia'].values
        self.transforms = transforms
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(Config['img_size'], Config['img_size']),
        A.HorizontalFlip(p=1.0),
        A.VerticalFlip(p=1.0),
        A.Rotate(limit=180, p=0.7),
        A.ShiftScaleRotate(
            shift_limit = 0.1, scale_limit=0.1, rotate_limit=45, p=0.5
        ),
#         A.HueSaturationValue(
#             hue_shift_limit=0.2, sat_shift_limit=0.2,
#             val_shift_limit=0.2, p=0.5
#         ),
#         A.RandomBrightnessContrast(
#         brightness_limit=(-0.1, 0.1),
#             contrast_limit=(-0.1, 0.1), p=0.5
#         ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(Config['img_size'], Config['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

In [ ]:
class block(nn.Module):
    def __init__(self, first_conv_in_channels, first_conv_out_channels, identity_conv=None, stride=1):
        """
        残差ブロックを作成するクラス
        Args:
            first_conv_in_channels : 1番目のconv層（1×1）のinput channel数
            first_conv_out_channels : 1番目のconv層（1×1）のoutput channel数
            identity_conv : channel数調整用のconv層
            stride : 3×3conv層におけるstide数。sizeを半分にしたいときは2に設定
        """        
        super(block, self).__init__()

        # 1番目のconv層（1×1）
        self.conv1 = nn.Conv2d(
            first_conv_in_channels, first_conv_out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(first_conv_out_channels)

        # 2番目のconv層（3×3）
        # パターン3の時はsizeを変更できるようにstrideは可変
        self.conv2 = nn.Conv2d(
            first_conv_out_channels, first_conv_out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(first_conv_out_channels)

        # 3番目のconv層（1×1）
        # output channelはinput channelの4倍になる
        self.conv3 = nn.Conv2d(
            first_conv_out_channels, first_conv_out_channels*4, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(first_conv_out_channels*4)
        self.relu = nn.ReLU()

        # identityのchannel数の調整が必要な場合はconv層（1×1）を用意、不要な場合はNone
        self.identity_conv = identity_conv

    def forward(self, x):

        identity = x.clone()  # 入力を保持する

        x = self.conv1(x)  # 1×1の畳み込み
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)  # 3×3の畳み込み（パターン3の時はstrideが2になるため、ここでsizeが半分になる）
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)  # 1×1の畳み込み
        x = self.bn3(x)

        # 必要な場合はconv層（1×1）を通してidentityのchannel数の調整してから足す
        if self.identity_conv is not None:
            identity = self.identity_conv(identity)
        x += identity

        x = self.relu(x)

        return x
    

class ResNet(nn.Module):
    def __init__(self, block, num_classes):
        super(ResNet, self).__init__()

        # conv1はアーキテクチャ通りにベタ打ち
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # conv2_xはサイズの変更は不要のため、strideは1
        self.conv2_x = self._make_layer(block, 3, res_block_in_channels=64, first_conv_out_channels=64, stride=1)

        # conv3_x以降はサイズの変更をする必要があるため、strideは2
        self.conv3_x = self._make_layer(block, 4, res_block_in_channels=256,  first_conv_out_channels=128, stride=2)
        self.conv4_x = self._make_layer(block, 6, res_block_in_channels=512,  first_conv_out_channels=256, stride=2)
        self.conv5_x = self._make_layer(block, 3, res_block_in_channels=1024, first_conv_out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(512*4, num_classes)
#         self.fc2 = nn.Linear(512*3, 512*2)
#         self.fc3 = nn.Linear(512*2, 512)
#         self.fc4 = nn.Linear(512, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):

        x = self.conv1(x)   # in:(3,224*224)、out:(64,112*112)
        x = self.bn1(x)     # in:(64,112*112)、out:(64,112*112)
        x = self.relu(x)    # in:(64,112*112)、out:(64,112*112)
        x = self.maxpool(x) # in:(64,112*112)、out:(64,56*56)

        x = self.conv2_x(x)  # in:(64,56*56)  、out:(256,56*56)
        x = self.conv3_x(x)  # in:(256,56*56) 、out:(512,28*28)
        x = self.conv4_x(x)  # in:(512,28*28) 、out:(1024,14*14)
        x = self.conv5_x(x)  # in:(1024,14*14)、out:(2048,7*7)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
#         x = self.fc4(x)
        x = self.sigmoid(x)

        return x

    def _make_layer(self, block, num_res_blocks, res_block_in_channels, first_conv_out_channels, stride):
        layers = []

        # 1つ目の残差ブロックではchannel調整、及びsize調整が発生する
        # identifyを足す前に1×1のconv層を追加し、サイズ調整が必要な場合はstrideを2に設定
        identity_conv = nn.Conv2d(res_block_in_channels, first_conv_out_channels*4, kernel_size=1,stride=stride)
        layers.append(block(res_block_in_channels, first_conv_out_channels, identity_conv, stride))

        # 2つ目以降のinput_channel数は1つ目のoutput_channelの4倍
        in_channels = first_conv_out_channels*4

        # channel調整、size調整は発生しないため、identity_convはNone、strideは1
        for i in range(num_res_blocks - 1):
            layers.append(block(in_channels, first_conv_out_channels, identity_conv=None, stride=1))

        return nn.Sequential(*layers)

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch, fold):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.float)
        labels = labels.unsqueeze(-1)
        
        batch_size = Config['batch_size']
        optimizer.zero_grad()
        outputs = model(images)
        sigmoid = nn.Sigmoid()
        outputs = sigmoid(outputs)
        criterion = nn.BCELoss()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
        
    gc.collect()
    
    return epoch_loss

In [ ]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch, fold):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        images = data['image'].to(device, dtype=torch.float)
        labels = data['label'].to(device, dtype=torch.float)
        labels = labels.unsqueeze(-1)

        batch_size = Config['batch_size']
        outputs = model(images)
        sigmoid = nn.Sigmoid()
        outputs = sigmoid(outputs)
        criterion = nn.BCELoss()
        loss = criterion(outputs, labels)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   

    gc.collect()
    
    return epoch_loss

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler,
                                           dataloader=train_loader, 
                                           device=Config['device'], epoch=epoch, fold=fold)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=Config['device'], 
                                         epoch=epoch, fold=fold)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            es = 0
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"{Config['model_name']}fold{fold}best.pt"
            torch.save(model.state_dict(), PATH)
        else:
            es += 1
            if es > 9:
                print("Early stopping with best_acc: ", best_epoch_loss)
                break

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    model.load_state_dict(best_model_wts)
        
    return model, history

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
        
    train_dataset = MyDataset(df_train, transforms=data_transforms["train"])
    valid_dataset = MyDataset(df_valid, transforms=data_transforms["valid"])

    train_loader = DataLoader(train_dataset, batch_size=Config['batch_size'], 
                              num_workers=0, shuffle=True, pin_memory=True, drop_last=True, worker_init_fn=seed_worker)
    valid_loader = DataLoader(valid_dataset, batch_size=Config['batch_size'], 
                              num_workers=0, shuffle=False, pin_memory=True, worker_init_fn=seed_worker)
    
    return train_loader, valid_loader

In [ ]:
for i in range(Config['n_fold']):
#     model = ResNet(block, 1)
    model = timm.create_model(Config['model_name'], pretrained=False, num_classes=1)
    model.to(Config['device'])
    train_loader, valid_loader = prepare_loaders(df_train, fold=i)
    optimizer = optim.Adam(model.parameters(), lr=Config['learning_rate'], weight_decay=Config['weight_decay'])
    scheduler = lr_scheduler.OneCycleLR(optimizer,max_lr =Config['learning_rate'],total_steps = Config['epochs'] * len(train_loader))
    model, history = run_training(model, optimizer, scheduler, device=Config['device'], num_epochs=Config['epochs'], fold=i)

In [ ]:
transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Rotate90(angles=[0, 90, 180]),
#         tta.Scale(scales=[1, 2, 4]),
#         tta.Multiply(factors=[0.9, 1, 1.1]),        
    ]
)

In [ ]:
def inference(test_dataloader, model):
    
    allpreds=[]

    total_loss = 0 # Initializing total loss

    model.eval()

    for a in test_dataloader:

        with torch.no_grad():
            images = a["image"].to(Config['device'])

            output = model(images) # prediction
            sigmoid = nn.Sigmoid()
            output = sigmoid(output)
            allpreds.append(output.detach().cpu().numpy())

    allpreds = np.concatenate(allpreds)
    
    return allpreds

In [ ]:
df_test['pneumonia'] = 0.5
test_dataset = MyDataset(df_test, transforms=data_transforms["valid"])
test_loader = DataLoader(test_dataset, batch_size=Config['batch_size'], 
                          num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
preds = []

In [ ]:
for i in range(Config['n_fold']):
    #     model = ResNet(block, 1)
    model = timm.create_model(Config['model_name'], pretrained=True, num_classes=1)
    model_name = f'./resnet18dfold{i}best.pt'
    model.load_state_dict(torch.load(model_name))
    model.to(Config['device'])
#     tta_model = tta.ClassificationTTAWrapper(model, transforms)
#     tta_model.to(Config['device'])
    pred = inference(test_loader, model)
    preds.append(pred)

In [ ]:
preds = np.array(preds)
average_preds = np.mean(preds, axis=0)
df_sub['pneumonia'] = average_preds
df_sub.to_csv('submission.csv', index=False)